In [1]:
import sys
cpath = !pwd
sys.path.append('/usr/src/app/algorithms/')
sys.path.append('/usr/src/app/')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts
from algorithms.anchor.anchor_tabular import AnchorTabularExplainer

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

glass = Dataset()

In [3]:
X, y, feature_names, class_names = glass.load_glass()
y = [int(i) for i in y]
y = np.array(y)
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn

In [4]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, iterr, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': ['sqrt'],
        'bootstrap': [True],
        'min_samples_leaf' : [2],
        'n_estimators': [1000]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task, random_state=10):
        
        
        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task, random_state)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150, random_state)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0, njobs=6, seed=random_state)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            length, nodes = mdl.find_rule_length(instances[0])
            max_len = len(length)
            comp = {1:'>',0:'<='}
            rules = {}
            for f in feature_names:
                rules[f] = []
            counter = 0
            for rule, node in list(zip(mdl.rule_,nodes)):
                for conj in range(len(node)):
                    if node[conj] and counter<=max_len-1:
                        rules[feature_names[int(rule[conj][0]-1)]].append([comp[int(rule[conj][1])],rule[conj][2]])
                        counter += 1
            new_rules = {}
            for k, v in rules.items():
                if len(v) == 1:
                    new_rules[k] = v
                else:
                    mmin = None
                    mmax = None
                    for value in v:
                        if value[0]=='<=':
                            if mmin is None or mmin > value[1]:
                                mmin = value[1]
                        if value[0]=='>':
                            if mmax is None or mmax < value[1]:
                                mmax = value[1]
                    if mmin is not None:
                        new_rules[k] = [['<=', mmin]]
                    if mmax is not None:
                        new_rules[k] = [['>', mmax]]
            te = time.time()
            #print(mdl)
            return length, cover, def_predictions, te-ts, new_rules
        
        #Anchors =======================================================================================
        explainer = AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95, random_state=random_state)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0
        #"""
        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_y = np.array([int(i+1) for i in y_train])
        temp_frame = pd.DataFrame(np.hstack((train,temp_y.reshape(len(temp_y),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'glass_'+str(iterr), random_state=random_state)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = False)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=random_state,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time()-chts)/len(test)
        
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte
        #"""
        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance, ar_algorithm='apriori', cl_algorithm='SC', instance_random_state=random_state)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
        #return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification', random_state)
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        #covered = False
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        #covered = False
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        #covered = False
                        return 0
        
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        #covered = False
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        #covered = False
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        #covered = False
                        return 0
        
        return 1
    
    #loo = LeaveOneOut()
    #loo.get_n_splits(test)
    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    rules = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    
    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)    
    
    for test_ind in range(len(test)):
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(test_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules['ch'].append(rule)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te, new_rules = method(np.concatenate((x_test_temp[test_ind:test_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
                rules['df'].append(new_rules)
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
                rules['lf'].append(rule)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules[name].append(rule)
            time_response[name].append(te)
            #print(rule)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response, rules

In [5]:
from sklearn.model_selection import StratifiedKFold
from collections import Counter

In [6]:
total_results2 = []
test_size_2 = []
for rand in [7, 10, 77]:
    total_results = []
    kf = StratifiedKFold(n_splits=10, random_state=777)
    folds = 0
    test_size = []
    for train_index, test_index in kf.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
        print(Counter(y_train), Counter(y_test))
        results = measure(X_train, X_test, y_train, y_test, feature_names, class_names, folds, random_state=rand)
        test_size.append(len(X_test))
        total_results.append(results)
        folds=folds+1
    test_size_2.append(test_size)
    total_results2.append(total_results)

# of Fold: 1, size of test: 22
Counter({1: 68, 0: 63, 5: 26, 2: 15, 3: 12, 4: 8}) Counter({1: 8, 0: 7, 5: 3, 2: 2, 3: 1, 4: 1})
len self.path_detail
22
Working on CHIRPS for instance 0 of 22
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
reduced 1 patterns out of 64 by numeric redundancy
found 63 patterns from 596 trees for batch_idx 0
start score sort for batch_idx 0 (63) patterns
start merge rule for batch_idx 0 (63) patterns
[('magnesium', False, 0.63623), ('potassium', True, 0.05979), ('refractive_index', False, 0.26203), ('aluminum', True, 0.41597), ('iron', True, 0.3914)]
0.7419354838709677 0.12364974478692675 0.1349073814600667 0.1852882089321588
merge complete for batch_idx 0 (63) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 54 patterns from 526 trees for batch_idx 1
start score sort for batch_idx 1 (54) patterns
start merge rule for batch_idx 1 (54) patterns
[('aluminum',

[('aluminum', False, 0.35004), ('aluminum', True, 0.40968), ('refractive_index', True, 0.26292)]
0.42857142857142855 0.039754327191513124 0.21633060218358705 0.2986683576410907
merge complete for batch_idx 16 (47) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
found 6 patterns from 336 trees for batch_idx 17
start score sort for batch_idx 17 (6) patterns
start merge rule for batch_idx 17 (6) patterns
[('barium', False, 0.0965), ('sodium', False, 0.46476)]
0.8064516129032258 0.12562814070351758 0.44486672064295885 0.5069582504970178
merge complete for batch_idx 17 (6) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
found 24 patterns from 553 trees for batch_idx 18
start score sort for batch_idx 18 (24) patterns
start merge rule for batch_idx 18 (24) patterns
[('aluminum', False, 0.41247), ('barium', True, 0.11164), ('potassium', True, 0.022

[('aluminum', False, 0.34618), ('refractive_index', True, 0.2587), ('magnesium', False, 0.67749)]
0.7941176470588235 0.1395406785996096 0.29051435366789163 0.2707939508506617
merge complete for batch_idx 10 (32) patterns
start get explainer for batch_idx 10
as_chirps for batch_idx 11
start mining for batch_idx 11 with support = 0.1
found 30 patterns from 451 trees for batch_idx 11
start score sort for batch_idx 11 (30) patterns
start merge rule for batch_idx 11 (30) patterns
[('magnesium', False, 0.72697), ('refractive_index', True, 0.26715), ('barium', True, 0.10231), ('calcium', True, 0.27187)]
0.7631578947368421 0.1568171435690851 0.22028096177417283 0.19747166521360074
merge complete for batch_idx 11 (30) patterns
start get explainer for batch_idx 11
as_chirps for batch_idx 12
start mining for batch_idx 12 with support = 0.1
found 30 patterns from 920 trees for batch_idx 12
start score sort for batch_idx 12 (30) patterns
start merge rule for batch_idx 12 (30) patterns
[('aluminum',

[('aluminum', False, 0.35011), ('magnesium', False, 0.75077)]
0.7692307692307693 0.16175137985007002 0.2324504950461197 0.2915590863952333
merge complete for batch_idx 5 (44) patterns
start get explainer for batch_idx 5
as_chirps for batch_idx 6
start mining for batch_idx 6 with support = 0.1
found 44 patterns from 576 trees for batch_idx 6
start score sort for batch_idx 6 (44) patterns
start merge rule for batch_idx 6 (44) patterns
[('aluminum', False, 0.35123), ('magnesium', False, 0.75143)]
0.7692307692307693 0.16175137985007002 0.17907377960454207 0.21214071214071215
merge complete for batch_idx 6 (44) patterns
start get explainer for batch_idx 6
as_chirps for batch_idx 7
start mining for batch_idx 7 with support = 0.1
found 35 patterns from 535 trees for batch_idx 7
start score sort for batch_idx 7 (35) patterns
start merge rule for batch_idx 7 (35) patterns
[('aluminum', False, 0.35058), ('magnesium', False, 0.73552), ('barium', True, 0.1243), ('calcium', True, 0.27525)]
0.805555

[('calcium', False, 0.28887), ('magnesium', False, 0.76169), ('iron', True, 0.38945), ('refractive_index', False, 0.27451)]
0.8333333333333334 0.17946877243359657 0.1445080232928089 0.14293806646525678
merge complete for batch_idx 0 (56) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 49 patterns from 897 trees for batch_idx 1
start score sort for batch_idx 1 (49) patterns
start merge rule for batch_idx 1 (49) patterns
[('magnesium', False, 0.58357), ('refractive_index', False, 0.2652), ('refractive_index', True, 0.30511), ('aluminum', True, 0.36643)]
0.7948717948717948 0.1631969370662838 0.27089313467870324 0.29199260418148204
merge complete for batch_idx 1 (49) patterns
start get explainer for batch_idx 1
as_chirps for batch_idx 2
start mining for batch_idx 2 with support = 0.1
found 49 patterns from 941 trees for batch_idx 2
start score sort for batch_idx 2 (49) patterns
start merge rule for batch_idx 2 (49

...

[('magnesium', False, 0.78725), ('aluminum', True, 0.33398), ('iron', True, 0.27076), ('refractive_index', False, 0.26179)]
0.7111111111111111 0.18433593750000002 0.16364433085175842 0.27386480908152727
merge complete for batch_idx 1 (54) patterns
start get explainer for batch_idx 1
as_chirps for batch_idx 2
start mining for batch_idx 2 with support = 0.1
found 44 patterns from 662 trees for batch_idx 2
start score sort for batch_idx 2 (44) patterns
start merge rule for batch_idx 2 (44) patterns
[('calcium', False, 0.27432), ('magnesium', False, 0.71856), ('magnesium', True, 0.85408), ('refractive_index', False, 0.26114)]
0.6875 0.25828124999999996 0.05737303425099941 0.0347016504443504
merge complete for batch_idx 2 (44) patterns
start get explainer for batch_idx 2
as_chirps for batch_idx 3
start mining for batch_idx 3 with support = 0.1
found 64 patterns from 879 trees for batch_idx 3
start score sort for batch_idx 3 (64) patterns
start merge rule for batch_idx 3 (64) patterns
[('mag

[('barium', False, 0.10479)]
0.7575757575757576 0.13338323353293413 0.3543753176050164 0.27611484422724497
merge complete for batch_idx 18 (9) patterns
start get explainer for batch_idx 18
as_chirps for batch_idx 19
start mining for batch_idx 19 with support = 0.1
found 9 patterns from 953 trees for batch_idx 19
start score sort for batch_idx 19 (9) patterns
start merge rule for batch_idx 19 (9) patterns
[('barium', False, 0.10168)]
0.7575757575757576 0.13338323353293413 0.2661768218576643 0.31675392670157065
merge complete for batch_idx 19 (9) patterns
start get explainer for batch_idx 19
Working on CHIRPS for instance 20 of 21
as_chirps for batch_idx 20
start mining for batch_idx 20 with support = 0.1
found 8 patterns from 997 trees for batch_idx 20
start score sort for batch_idx 20 (8) patterns
start merge rule for batch_idx 20 (8) patterns
[('barium', False, 0.11116)]
0.7575757575757576 0.13338323353293413 0.4373745810076021 0.33484162895927605
merge complete for batch_idx 20 (8) p

[('magnesium', False, 0.74519), ('calcium', False, 0.28686), ('iron', True, 0.42675), ('refractive_index', False, 0.2614)]
0.7647058823529411 0.21461538461538462 0.17817312271266583 0.21549079754601236
merge complete for batch_idx 15 (36) patterns
start get explainer for batch_idx 15
as_chirps for batch_idx 16
start mining for batch_idx 16 with support = 0.1
reduced 1 patterns out of 27 by numeric redundancy
found 26 patterns from 418 trees for batch_idx 16
start score sort for batch_idx 16 (26) patterns
start merge rule for batch_idx 16 (26) patterns
[('magnesium', True, 0.56152), ('sodium', True, 0.43016), ('aluminum', False, 0.33827)]
0.6 0.06922651933701658 0.41147338129634664 0.38519259629814906
merge complete for batch_idx 16 (26) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
found 21 patterns from 388 trees for batch_idx 17
start score sort for batch_idx 17 (21) patterns
start merge rule for batch_idx 17

start merge rule for batch_idx 12 (41) patterns
[('aluminum', False, 0.34996), ('magnesium', False, 0.56616), ('refractive_index', True, 0.26694)]
0.7608695652173914 0.19186991869918701 0.21608677290690592 0.2815374507227333
merge complete for batch_idx 12 (41) patterns
start get explainer for batch_idx 12
as_chirps for batch_idx 13
start mining for batch_idx 13 with support = 0.1
found 35 patterns from 789 trees for batch_idx 13
start score sort for batch_idx 13 (35) patterns
start merge rule for batch_idx 13 (35) patterns
[('aluminum', False, 0.34984), ('magnesium', False, 0.54565), ('refractive_index', True, 0.26643)]
0.7608695652173914 0.19186991869918701 0.243131392923665 0.3245410424140113
merge complete for batch_idx 13 (35) patterns
start get explainer for batch_idx 13
as_chirps for batch_idx 14
start mining for batch_idx 14 with support = 0.1
found 38 patterns from 406 trees for batch_idx 14
start score sort for batch_idx 14 (38) patterns
start merge rule for batch_idx 14 (38)

[('aluminum', False, 0.35258), ('magnesium', False, 0.5468), ('refractive_index', True, 0.26268)]
0.75 0.18227642276422765 0.21820217141492718 0.23462664714494885
merge complete for batch_idx 8 (34) patterns
start get explainer for batch_idx 8
as_chirps for batch_idx 9
start mining for batch_idx 9 with support = 0.1
reduced 5 patterns out of 49 by numeric redundancy
found 44 patterns from 448 trees for batch_idx 9
start score sort for batch_idx 9 (44) patterns
start merge rule for batch_idx 9 (44) patterns
[('iron', False, 0.23122), ('magnesium', False, 0.5829)]
0.4722222222222222 0.13414634146341461 0.02607983869970503 0.013675690761931343
merge complete for batch_idx 9 (44) patterns
start get explainer for batch_idx 9
Working on CHIRPS for instance 10 of 21
as_chirps for batch_idx 10
start mining for batch_idx 10 with support = 0.1
found 40 patterns from 485 trees for batch_idx 10
start score sort for batch_idx 10 (40) patterns
start merge rule for batch_idx 10 (40) patterns
[('alumi

In [9]:
from utilities.lionforests_utility import path_similarity
rule_variance = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}


kf = StratifiedKFold(n_splits=10, random_state=777)
folds = 0
test_size = []
for train_index, test_index in kf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    min_max_feature_values = {}
    for i in range(len(feature_names)):
            min_max_feature_values[feature_names[i]] = [min(X_train[:, i]), max(X_train[:, i])] 
    for name in ['gs', 'ls', 'an', 'lf', 'df', 'ch']:
        for k in range(len(total_results2[0][folds][-1][name])):
            r1 = total_results2[0][folds][-1][name][k]
            r2 = total_results2[1][folds][-1][name][k]
            r3 = total_results2[2][folds][-1][name][k]
            rule_variance[name].append((path_similarity(r1, r2, feature_names, min_max_feature_values)+
                                           path_similarity(r1, r3, feature_names, min_max_feature_values)+
                                           path_similarity(r2, r3, feature_names, min_max_feature_values))/3)
    folds = folds + 1

In [10]:
f_full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
f_f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
for b in range(3):
    full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    k = 0
    for i in total_results2[b]:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/test_size_2[b][k])
            rule_length[name].append(i[2][name]/test_size_2[b][k])
            f_precision[name].append(np.array(i[3][name]).mean())
            f_time[name].append(np.array(i[4][name]).mean())
        k = + 1
    for name, method in i[0].items():
        f_full_coverage[name].append(np.array(full_coverage[name]).mean())
        f_rule_length[name].append(np.array(rule_length[name]).mean())
        f_f_precision[name].append(np.array(f_precision[name]).mean())
        f_f_time[name].append(np.array(f_time[name]).mean())
for name, method in i[0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()))

gs | 0.1425  0.012 | 4.5303 0.267 | 0.8248  0.028 | 0.0003  0.000 | -0.3474  7.045
ls | 0.1488  0.003 | 3.8121 0.053 | 0.8028  0.004 | 2.1683  0.022 | 0.4209  0.791
an | 0.0836  0.001 | 5.6924 0.058 | 0.3699  0.007 | 90.3071  0.698 | -0.5024  5.090
lf | 0.0441  0.000 | 8.2318 0.000 | 1.0000  0.000 | 4.6880  0.013 | 0.9290  0.090
df | 0.9545  0.011 | 1.8288 0.231 | 0.4597  0.011 | 0.3645  0.002 | 0.7723  0.121
ch | 0.2776  0.002 | 2.6924 0.031 | 0.8316  0.002 | 0.5643  0.002 | 0.8798  0.098


In [13]:
import csv  

with open('glass.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    for name, method in i[0].items():
        writer.writerow([name, np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()])